<a href="https://colab.research.google.com/github/Samir-atra/CancerDetector/blob/main/Cancer_Detector/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Imports

!pip install keras_tuner -q
import tensorflow as tf
import keras_tuner
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import sys
import time

In [1]:
# Mounting google drive for training in google co lab

from google.colab import drive             
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')

data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
AUTOTUNE=tf.data.AUTOTUNE

start = time.time()
dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(      # Testing dataset
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

end = time.time()
print("time elapsed = ", (end - start))

Found 8582 files belonging to 4 classes.
Using 6866 files for training.
Found 8582 files belonging to 4 classes.
Using 1716 files for validation.
Found 1705 files belonging to 4 classes.
time elapsed =  31.64029574394226


In [6]:
# catch and prefetch for shorter training time

# AUTOTUNE=tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
# downloading the weights oft the base model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = False, 
    weights = "imagenet"                  
)

87910968/87910968 [==============================] - 1s 0us/step


In [8]:
#saving the downloaded base_model 

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model.save(saving_path)

In [ ]:
#Model

model_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([                                        
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp):

inputs = tf.keras.Input(shape=(299, 299, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Dense(1024, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)     # Neurons numbers and activation functions based on testing and choosing the one with best results
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(512, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)     # Regularizing and Dropout to avoid overfitting
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(512, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(256, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)

# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00050233),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)                                               
  
#  return model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Transfer",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val)

# tuner.results_summary()


model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=15,   
    validation_data = dataset_path_val)
    #verbose = 1)


model.evaluate(dataset_path_test, batch_size=5, verbose=1)               # evaluating using the test dataset 

model.summary()

base_model.trainable = True                                           # switching the base_model to trainable

for layer in base_model.layers:                                       # Swtching all batchnormalization layer of the base_model to non-trainable to not lose weights
  if isinstance(layer, tf.keras.layers.BatchNormalization):
    layer.trainable = False

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001),    # compiling the model with a trainable base_model
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

# model.fit(                                                            # fitting for another 10 epochs starting where the non-trainable ended
#     dataset_path,
#     epochs=30,         
#     initial_epoch=15,                
#     validation_data = dataset_path_val)
    #verbose = 1)

model.evaluate(dataset_path_test, batch_size=5, verbose=1)

model.summary()

# saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedModelWithFT/')

# tf.keras.models.save_model(model,                                     # saving the fully trained model
#                            saving_path,
#                            overwrite=True,
#                            save_format='tf'
#                            )

Trial 2 Complete [01h 23m 40s]
val_accuracy: 0.9111305177211761

Best val_accuracy So Far: 0.9111305177211761
Total elapsed time: 02h 58m 28s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
3.9869e-05        |0.00050233        |lr

Epoch 1/5
1374/1374 [==============================] - 523s 380ms/step - loss: 1.0540 - accuracy: 0.5942 - val_loss: 0.8026 - val_accuracy: 0.7436
Epoch 2/5
1374/1374 [==============================] - 501s 365ms/step - loss: 0.7079 - accuracy: 0.7716 - val_loss: 0.6376 - val_accuracy: 0.7995
Epoch 3/5
1374/1374 [==============================] - 497s 362ms/step - loss: 0.5919 - accuracy: 0.8051 - val_loss: 0.5623 - val_accuracy: 0.8199
Epoch 4/5
1374/1374 [==============================] - 496s 361ms/step - loss: 0.5318 - accuracy: 0.8248 - val_loss: 0.5164 - val_accuracy: 0.8322
Epoch 5/5
1374/1374 [==============================] - 490s 357ms/step - loss: 0.4933 - accuracy: 0.8392 - val_loss: 0.4837 - val_accuracy: 0.8438
E